In [1]:
import numpy as np
import pandas as pd
from sklearn import datasets, metrics
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier

import os
print(os.listdir("../input"))

['test.csv', 'trainLabels.csv', 'train.csv']


In [2]:
data=pd.read_csv("../input/train.csv")
target=pd.read_csv("../input/trainLabels.csv")

# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(data, target, train_size=0.7,random_state=42)

# 建立模型
clf = GradientBoostingClassifier(random_state=7)

/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
n_estimators = range(10, 500, 5)
max_depth = range(1, 100, 2)
param_grid = dict(n_estimators=n_estimators, max_depth=max_depth)
# param_grid = dict(n_estimators=n_estimators)

## 建立搜尋物件，放入模型及參數組合字典 (n_jobs=-1 會使用全部 cpu 平行運算)
grid_search = GridSearchCV(
    clf, param_grid, scoring="neg_mean_squared_error", n_jobs=-1, verbose=1)

# 開始搜尋最佳參數
grid_result = grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 4900 candidates, totalling 14700 fits


/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.6s
[Parallel(n_jobs=-1)]: Done 323 tasks      | elapsed:   12.3s
[Parallel(n_jobs=-1)]: Done 626 tasks      | elapsed:   44.4s
[Parallel(n_jobs=-1)]: Done 976 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1426 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 1976 tasks      | elapsed:  3.3min
[Parallel(n_jobs=-1)]: Done 2626 tasks      | elapsed:  4.5min
[Parallel(n_jobs=-1)]: Done 3376 tasks      | elapsed:  5.7min
[Parallel(n_jobs=-1)]: Done 4226 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 5176 tasks      | elapsed:  8.9min
[Parallel(n_job

In [4]:
print("Best Accuracy: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best Accuracy: -0.134478 using {'max_depth': 5, 'n_estimators': 145}


In [5]:
grid_result.best_params_

{'max_depth': 5, 'n_estimators': 145}

In [6]:
# 使用最佳參數重新建立模型
clf_bestparam = GradientBoostingClassifier(max_depth=grid_result.best_params_['max_depth'],
                                           n_estimators=grid_result.best_params_['n_estimators'])
# 訓練模型
clf_bestparam.fit(x_train, y_train)
clf_bestparam.score(x_test, y_test)

/anaconda3/envs/venv/lib/python3.7/site-packages/sklearn/utils/validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.8433333333333334

In [7]:
test = pd.read_csv("../input/test.csv", header=None)
y_pred = clf_bestparam.predict(test)
index_column = np.arange(1, y_pred.shape[0] + 1)
data_submission = pd.DataFrame(y_pred, columns=['Solution'], dtype=int)
data_submission['Id'] = index_column
data_submission.index.name = 'Id'
data_submission.set_index(index_column)

,Solution,Id
1,1,1
2,0,2
3,1,3
4,0,4
5,0,5
6,0,6
7,0,7
8,1,8
9,0,9
10,0,10


In [8]:
data_submission.to_csv("submission.csv", index=False)